In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.optimizers import Adam

In [3]:
# ACF+ASA+AAINDEX+BTA+CKSAAP+GPS+OBC+PSEAAC+PSSM+SS
# 610  61   610   244   411  300 1342   20  1220 183


def get_col(fn):
    finame = r'./results/10features/' + str.upper(fn) + r'/A0A0A0MXU4_212.' + str.lower(fn)
    with open(finame) as file:
        values = file.read().rstrip().split('\t')
        i = 0
        columns = []
        while i < len(values):
            columns.append(str(i))
            i += 1
    return columns, len(values)


def prep_dataset(feature_name):
    global dataset
    col, size = get_col(feature_name)
    x = dataset[col]
    y = dataset['label']
    ID = dataset['pepname']
    return x, y, ID, size


Hidden1_size = {'PSEAAC': 2048, 'CKSAAP': 2048, 'OBC': 64, 'AAINDEX': 1024, 'ACF': 2048, 'GPS': 512, 'PSSM': 1024,
                'ASA': 2048, 'SS': 2048, 'BTA': 2048, 'transformer': 2048}
Hidden2_size = {'PSEAAC': 1024, 'CKSAAP': 1024, 'OBC': 32, 'AAINDEX': 512, 'ACF': 1024, 'GPS': 256, 'PSSM': 512,
                'ASA': 1024, 'SS': 1024, 'BTA': 1024, 'transformer': 1024}


def get_model(size):
    global dropout_rate, feature
    from tensorflow.keras.layers import Dense
    from tensorflow.keras import Sequential
    dnn = Sequential()
    dnn.add(Dense(size, input_shape=(size,), bias_initializer='ones', name='Input'))
    dnn.add(Dense(Hidden1_size[feature], activation='relu', name='Hidden1'))
    dnn.add(layers.Dropout(dropout_rate[feature], name='Dropout1'))
    dnn.add(Dense(Hidden2_size[feature], activation='relu', name='Hidden2'))
    dnn.add(layers.Dropout(dropout_rate[feature], name='Dropout2'))
    dnn.add(Dense(1, activation='sigmoid', name='Output'))
    dnn.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['acc'])
    return dnn


def training_DNN(feature_name):
    print('\n')
    print('————————————' + feature_name + '————————————')
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import StratifiedKFold

    x, y, pepID, feature_size = prep_dataset(feature_name)
    x = x.values
    y = y.values
    pepID = pepID.values

    skf = StratifiedKFold(n_splits=5, shuffle=False)
    count = 1
    y_label = []
    y_score = []
    # models = []
    peplist = []

    for train_index, test_index in skf.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]

        dnn = get_model(feature_size)
        dnn.fit(x_train, y_train, epochs=15, batch_size=4096)

        y_label.append(list(y_test))
        y_test_score = dnn.predict(x_test)
        y_score.append(list(y_test_score))
        # models.append(dnn)
        dnn.save(r'./models/DNN/' + feature_name + r'/' + feature_name + r'_DNN_' + str(count) +
                 r'.h5')
        peplist.append(list(pepID[test_index]))

        auc_score = roc_auc_score(y_test, y_test_score)
        print('第', count, '个', feature_name, '模型的AUC分数为：', auc_score)
        count += 1
        # reset_keras()

    # for j, model in enumerate(models):  # 保存十个模型
    # model.save(r'./models/DNN/' + str.upper(feature_name) + r'/' + str.upper(feature_name) + r'_DNN_' + str(j) +
    # r'.h5')
    from itertools import chain
    df_y = pd.concat([pd.DataFrame(list(chain.from_iterable(peplist)), columns=['pepname']),
                      pd.DataFrame(list(chain.from_iterable(y_label)), columns=['label']),
                      pd.DataFrame(list(chain.from_iterable(y_score)), columns=['score'])], axis=1)
    df_y.to_csv(r'./models/DNN/' + feature_name + r'/' + feature_name + r'_y_label&score.csv', index=False)

    AUC_score = roc_auc_score(list(chain.from_iterable(y_label)), list(chain.from_iterable(y_score)))
    print('————————————', feature_name, '模型的最终AUC分数为：', AUC_score, '————————————')
    return AUC_score

In [4]:
#features = ['ACF', 'ASA', 'AAINDEX', 'BTA', 'CKSAAP', 'GPS', 'OBC', 'PSEAAC', 'PSSM', 'SS']
#features = ['transformer']
features = ['GPS']
feature_scores = []
dropout_rate = {'ACF': 0.1, 'ASA': 0.2, 'AAINDEX': 0.1, 'BTA': 0.2, 'CKSAAP': 0.5, 'GPS': 0.2, 'OBC': 0.3,
                'PSEAAC': 0.5, 'PSSM': 0.2, 'SS': 0.2, 'transformer': 0.5}
for feature in features:
    dataset = pd.read_csv(r'./results/' + feature + r'_dataset1.csv')
    feature_scores.append(training_DNN(feature))



————————————GPS————————————
Epoch 1/15


C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.5079 - loss: 0.8255
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5148 - loss: 0.7193
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5331 - loss: 0.7023
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5366 - loss: 0.6929
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5431 - loss: 0.6829
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5511 - loss: 0.6772
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5547 - loss: 0.6712
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5546 - loss: 0.6695
Epoch 9/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5616 - loss: 0.6637
Epoch 10/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5639 - loss: 0.6621
Epoch 11/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5675 - loss: 0.6586
Epoch 12/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5687 - loss: 0.6568
Epoch 13/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/s

C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


第 1 个 GPS 模型的AUC分数为： 0.6020475713587768
Epoch 1/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.5142 - loss: 0.7135
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5373 - loss: 0.6919
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5476 - loss: 0.6800
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5559 - loss: 0.6715
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5648 - loss: 0.6652
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5657 - loss: 0.6626
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5739 - loss: 0.6590
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5800 - loss: 0.6557
Epoch 9/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5870 - loss: 0.6512
Epoch 10/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5859 - loss: 0.6504
Epoch 11/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5916 - loss: 0.6487
Epoch 12/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5905 - loss: 0.64

...

Epoch 1/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.4943 - loss: 0.9338
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5066 - loss: 0.7510
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5069 - loss: 0.7229
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5091 - loss: 0.7094
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5131 - loss: 0.7040
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5282 - loss: 0.6954
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5313 - loss: 0.6922
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5377 - loss: 0.6885
Epoch 9/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5380 - loss: 0.6869
Epoch 10/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5366 - loss: 0.6844
Epoch 11/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5448 - loss: 0.6791
Epoch 12/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5534 - loss: 0.6769
Epoch 13/15
10/10 ━━━━━━━━━━━━━━━━━━━

第 4 个 GPS 模型的AUC分数为： 0.6693618610442496


C:\Users\Administrator\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - acc: 0.4943 - loss: 0.9609
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.4991 - loss: 0.7607
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5120 - loss: 0.7107
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5213 - loss: 0.6963
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5326 - loss: 0.6892
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5474 - loss: 0.6796
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5501 - loss: 0.6775
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5505 - loss: 0.6727
Epoch 9/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5541 - loss: 0.6688
Epoch 10/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.5614 - loss: 0.6663
Epoch 11/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5670 - loss: 0.6631
Epoch 12/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - acc: 0.5738 - loss: 0.6570
Epoch 13/15
10/10 ━━━━━━━━━━━━━━━━━━━

第 5 个 GPS 模型的AUC分数为： 0.601229094021932
———————————— GPS 模型的最终AUC分数为： 0.6388612083462765 ————————————
